In [1]:
from skimage.feature import graycomatrix, graycoprops
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern
from sklearn import decomposition
from sklearn import metrics
from PIL import Image
import pandas as pd
import numpy as np
import os

newSize = 128 # Dimensão 128, 64, 32, 16, 8
dataDir = './data'

c:\ferramentass\Python3.9\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ferramentass\Python3.9\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\ferramentass\Python3.9\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def doLBP(ImageArray):
    lbp = local_binary_pattern(ImageArray, 3, 8 * 3)
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins))

    return hist.tolist()

def doGLCM(ImageArray, angles_range):
    glcmMatrix = np.empty((0,len(angles_range)),float)
    glcm = graycomatrix(ImageArray, distances=range(0,100,2), angles=angles_range, levels=256, symmetric=True, normed=True)
    glcmContrast = graycoprops(glcm, 'contrast')
    glcmDissimilarity = graycoprops(glcm, 'dissimilarity')
    glcmHomogeneity = graycoprops(glcm, 'homogeneity')
    glcmASM = graycoprops(glcm, 'ASM')
    glcmEnergy = graycoprops(glcm, 'energy')
    glcmCorrelation = graycoprops(glcm, 'correlation')

    glcmMatrix = np.append(glcmMatrix, glcmContrast, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmDissimilarity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmHomogeneity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmASM, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmEnergy, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmCorrelation, axis=0)

    return glcmMatrix.flatten().tolist()

def doPCA(dataFrame, newSize=128):
    y = dataFrame[0]
    X = dataFrame.drop(0, axis=1).values

    pca = decomposition.PCA(n_components=newSize)
    pca.fit(X)

    XPCA = pca.transform(X)

    dataFramePCA = pd.DataFrame(XPCA)
    dataFramePCA['Classe'] = y
    dataFramePCA = dataFramePCA[['Classe'] + dataFramePCA.columns[:-1].tolist()]
    
    return dataFramePCA

def doKnn(X_train, y_train, X_teste, y_teste, MAX_K=30):
    range_k = np.arange(1, MAX_K, 1)
    scores = {}

    for k in range_k:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_teste)

        scores[k] = metrics.accuracy_score(y_teste, y_pred)
    
    return scores, range_k

In [3]:
maxH = 2073
maxW = 2339
dataTeste = []
dataTreino = []

angle_step = np.pi/4
angle_end = np.pi
angles_range = np.arange(0, angle_end, angle_step)

for i, dir in enumerate(os.listdir(dataDir)[:10]):
    
    # Lista todas as imagens do direttório
    images = os.listdir(f'{dataDir}/{dir}/')

    # Abrir as imagens de treino e teste
    ImageDefaultTreino = Image.open(f'{dataDir}/{dir}/{images[0]}')
    ImageDefaultTeste = Image.open(f'{dataDir}/{dir}/{images[1]}')

    '''
    Fluxo de Treino
    '''

    # Redimensiona a imagem para 2339x2073
    ImageResize = ImageDefaultTreino.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    # Aplica o LBP
    lbp = doLBP(ImageArray)

    # Aplica o GLCM
    glcm = doGLCM(ImageArray, angles_range)

    # Adiciona os valores na lista
    dataTreino.append([i] + lbp + glcm)

    '''
    Fluxo de Teste
    '''

    # Redimensiona a imagem para 2339x2073
    ImageResize = ImageDefaultTeste.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    # Aplica o LBP
    lbp = doLBP(ImageArray)

    # Aplica o GLCM
    glcm = doGLCM(ImageArray, angles_range)
    
    # Adiciona os valores na lista
    dataTeste.append([i] + lbp + glcm)

In [ ]:
dataFrameTreino = pd.DataFrame(dataTreino)
dataFrameTeste = pd.DataFrame(dataTeste)

dataFrameTreino.to_csv('data_treino.csv', index=False)
dataFrameTeste.to_csv('data_teste.csv', index=False)

In [ ]:
PCATreino = doPCA(dataFrameTreino, 10)
PCATeste = doPCA(dataFrameTeste, 10)

In [ ]:
scores, range_k = doKnn(PCATreino.drop(['Classe'], axis=1).values, PCATreino['Classe'].values, PCATeste.drop(['Classe'], axis=1).values, PCATeste['Classe'].values)

In [ ]:
scores

{1: 0.18,
 2: 0.16,
 3: 0.12,
 4: 0.14,
 5: 0.16,
 6: 0.12,
 7: 0.08,
 8: 0.1,
 9: 0.08,
 10: 0.06,
 11: 0.04,
 12: 0.04,
 13: 0.04,
 14: 0.04,
 15: 0.04,
 16: 0.02,
 17: 0.02,
 18: 0.02,
 19: 0.02,
 20: 0.02,
 21: 0.02,
 22: 0.02,
 23: 0.02,
 24: 0.02,
 25: 0.02,
 26: 0.02,
 27: 0.02,
 28: 0.02,
 29: 0.02}